In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.model_selection import cross_val_score

In [3]:
df = pd.read_csv('newdata.csv')

In [4]:
df_cleaned = df.drop(columns=['Url', 'Country', 'Province', 'Region'])

In [5]:
risk_levels = {'NO_DATA': 'NON_FLOOD_ZONE'}

df['FloodingZone'] = df['FloodingZone'].replace(risk_levels)

df_encoded = pd.get_dummies(df_cleaned, columns=['FloodingZone', 'TypeOfSale'])
df

,Url,BathroomCount,BedroomCount,ConstructionYear,Country,District,Fireplace,FloodingZone,Furnished,Garden,...,SwimmingPool,Terrace,ToiletCount,TypeOfProperty,TypeOfSale,SubtypeOfProperty_Numerical,TypeOfSale_Numerical,Kitchen_Numerical,StateOfBuilding_Numerical,FloodingZone_Numerical
0,https://www.immoweb.be/en/classified/apartment...,1.0,1.0,1969.000000,Belgium,5,0.0,NON_FLOOD_ZONE,0.0,0.0,...,0.0,1.0,1.0,2,residential_sale,7,0,0,4,0
1,https://www.immoweb.be/en/classified/house/for...,6.0,13.0,1920.000000,Belgium,37,0.0,NON_FLOOD_ZONE,0.0,0.0,...,0.0,0.0,5.0,1,residential_sale,6,0,0,4,0
2,https://www.immoweb.be/en/classified/house/for...,2.0,4.0,2008.000000,Belgium,5,0.0,NON_FLOOD_ZONE,1.0,0.0,...,0.0,0.0,2.0,1,residential_sale,2,0,3,4,1
3,https://www.immoweb.be/en/classified/house/for...,1.0,4.0,1987.240844,Belgium,40,0.0,NON_FLOOD_ZONE,0.0,1.0,...,0.0,1.0,2.0,1,residential_sale,2,0,0,3,0
4,https://www.immoweb.be/en/classified/apartment...,0.0,2.0,1972.000000,Belgium,14,0.0,NON_FLOOD_ZONE,0.0,0.0,...,0.0,1.0,1.0,2,residential_sale,1,0,0,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118695,https://www.immoweb.be/en/classified/house/for...,1.0,3.0,2024.000000,Belgium,12,0.0,NON_FLOOD_ZONE,0.0,1.0,...,0.0,0.0,0.0,1,residential_sale,2,0,0,4,0
118696,https://www.immoweb.be/en/classified/house/for...,4.0,4.0,1987.240844,Belgium,1,0.0,NON_FLOOD_ZONE,0.0,0.0,...,0.0,0.0,0.0,1,residential_sale,6,0,0,6,0
118697,https://www.immoweb.be/en/classified/apartment...,1.0,2.0,1987.240844,Belgium,1,0.0,NON_FLOOD_ZONE,0.0,0.0,...,0.0,1.0,1.0,2,residential_sale,1,0,4,4,1
118698,https://www.immoweb.be/en/classified/apartment...,1.0,2.0,1987.240844,Belgium,5,0.0,POSSIBLE_FLOOD_ZONE,0.0,0.0,...,0.0,1.0,0.0,2,residential_sale,1,0,0,0,2


In [6]:
df.to_csv('newdatalast.csv', index=False) 

In [21]:
from sklearn.model_selection import train_test_split

X = df_encoded.drop('Price', axis=1)  
y = df_encoded['Price']              

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [14]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5

print("RMSE:", rmse)


RMSE: 239646.25576271492


In [22]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR

# create different models
models = {
    'RandomForest': RandomForestRegressor(n_estimators=100, random_state=42),
    'LinearRegression': LinearRegression(),
    'GradientBoosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'SVR': SVR()
}

#  teach them and test
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = mse ** 0.5
    print(f'{name} RMSE: {rmse}')


RandomForest RMSE: 239646.25576271492
LinearRegression RMSE: 6668664.239559636
GradientBoosting RMSE: 278568.5418599403
SVR RMSE: 457150.9250303294


In [23]:
from catboost import CatBoostRegressor 
from sklearn.metrics import mean_absolute_error
model = CatBoostRegressor()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mae

Learning rate set to 0.084068
0:	learn: 464223.5193058	total: 19.3ms	remaining: 19.3s
1:	learn: 449453.2395078	total: 34.4ms	remaining: 17.2s
2:	learn: 433726.0303644	total: 62.5ms	remaining: 20.8s
3:	learn: 421305.0072749	total: 84.8ms	remaining: 21.1s
4:	learn: 409943.2333122	total: 104ms	remaining: 20.7s
5:	learn: 399969.1508490	total: 122ms	remaining: 20.2s
6:	learn: 389162.7726529	total: 143ms	remaining: 20.3s
7:	learn: 380026.1065085	total: 162ms	remaining: 20.1s
8:	learn: 371652.4867341	total: 176ms	remaining: 19.4s
9:	learn: 363951.7584330	total: 194ms	remaining: 19.2s
10:	learn: 355379.6137520	total: 214ms	remaining: 19.3s
11:	learn: 346862.2160128	total: 238ms	remaining: 19.6s
12:	learn: 341163.5686232	total: 258ms	remaining: 19.6s
13:	learn: 334133.8186976	total: 289ms	remaining: 20.4s
14:	learn: 329145.9246336	total: 314ms	remaining: 20.6s
15:	learn: 324458.8638659	total: 328ms	remaining: 20.2s
16:	learn: 320248.4374221	total: 347ms	remaining: 20.1s
17:	learn: 316798.875500

81829.23027633599